In [1]:
import pandas as pd

In [12]:
df = pd.read_excel('Neocis01.xlsx')
df

,Tingresso,Solvente,Nd/PB,Al/Nd,S/Bde,Mooney,VEI
0,29.4663,30003.6,1.96452,5.88494,6.00264,41.2,0.91
1,29.3233,30008.8,1.96855,5.87461,6.00106,40.8,0.91
2,29.2119,29999.2,1.98207,5.82938,5.99960,38.8,0.92
3,29.2523,30012.4,1.98238,5.79447,6.00642,38.6,0.88
4,32.1935,30009.8,1.98835,5.79098,6.00157,37.3,0.88
5,34.1199,30028.9,1.83269,6.03725,6.00526,41.0,0.86
6,30.5015,29966.8,1.78801,5.71205,5.99352,41.3,0.87
7,30.2093,30003.6,1.78309,5.80321,5.99622,44.0,0.88
8,29.9590,29984.1,1.77951,5.68185,5.99628,43.0,0.85
9,29.8084,30020.0,1.70679,5.99015,6.00296,42.9,0.90
